In [175]:
from struct import unpack
import gzip
import numpy as np
from tqdm import tqdm_notebook
import scipy.misc as spm
from IPython.display import Image
from IPython.display import display

In [176]:
def get_labeled_data(imagefile, labelfile, lim):
    """Read input-vector (image) and target class (label, 0-9) and return
       it as list of tuples.
    """
    # Open the images with gzip in read binary mode
    images = gzip.open(imagefile, 'rb')
    labels = gzip.open(labelfile, 'rb')

    # Read the binary data

    # We have to get big endian unsigned int. So we need '>I'

    # Get metadata for images
    images.read(4)  # skip the magic_number
    number_of_images = images.read(4)
    number_of_images = unpack('>I', number_of_images)[0]
    rows = images.read(4)
    rows = unpack('>I', rows)[0]
    cols = images.read(4)
    cols = unpack('>I', cols)[0]

    # Get metadata for labels
    labels.read(4)  # skip the magic_number
    N = labels.read(4)
    N = unpack('>I', N)[0]

    # Get the data
    x = np.zeros((lim, rows, cols), dtype=np.float32)  # Initialize numpy array
    y = np.zeros((lim, 1), dtype=np.uint8)  # Initialize numpy array
    for i in tqdm_notebook(range(lim)):
        for row in range(rows):
            for col in range(cols):
                tmp_pixel = images.read(1)  # Just a single byte
                tmp_pixel = unpack('>B', tmp_pixel)[0]
                x[i][row][col] = tmp_pixel
        tmp_label = labels.read(1)
        y[i] = unpack('>B', tmp_label)[0]
    return (x, y)

(Images, Data) = get_labeled_data("train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz", 500)

In [177]:
def draw_image(image):
    img = np.copy(image)
    img[img == -1] = 0
    img = spm.toimage(img)
    return img

In [178]:
if np.max(Images == 255):
    Images /=255
    Images = np.around(Images, 0)
    Images[Images==0] = -1
    print(np.min(Images), np.max(Images))
    Noisy = np.copy(Images)

-1.0 1.0


In [179]:
#flip with 2% rate
for i in range(Images.shape[0]):
    for j in range(Images[i].shape[0]):
        for k in range(Images[i].shape[1]):
            if (np.random.rand() < .02):
                Noisy[i][j][k] *= -1


In [180]:
display(draw_image(Images[0]))
display(draw_image(Noisy[0]))